# Extract 8 regions

We're going to extract the 8 regions of interest (ROIs) from the _cis-eQTL_ data. 

## Load packages

Here we load the necessary packages.

In [1]:
# Function to check for installation of required packages
def check_install_package(package_name):
    try:
        importlib.import_module(package_name)
    except ImportError:
        print(f'{package_name} is not installed. Install and try again...')
        # subprocess.check_call(['pip', 'install', package_name])

import os
import glob
import importlib
import subprocess
import sys
# argument parsing
import argparse

# get date and time
from datetime import datetime

# Pandas is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool
# check_install_package('pandas')
import pandas as pd

# pyarrow is supperior to loading parquet files
# check_install_package('pyarrow')
import pyarrow as pa
import pyarrow.parquet as pq

# polars is a fast dataframe library
# check_install_package('polars')
import polars as pl

# for statistical analysis
# check_install_package('scipy')
from scipy import stats
import numpy as np

# scientific colourmaps
# https://www.fabiocrameri.ch/ws/media-library/8c4b111121ff448e843dfef9220bf613/readme_scientificcolourmaps.pdf
# check_install_package('cmcrameri')
import cmcrameri as ccm
from cmcrameri import cm

# for plotting
# check_install_package('matplotlib')
import matplotlib
import matplotlib.pyplot as plt
# if using a Jupyter notebook, include:
%matplotlib inline

# use Seaborn for visualisations
# check_install_package('seaborn')
import seaborn as sns

# for handling GWAS data
import gwaslab as gl

We also want some variables set. 

In [2]:
# Create directories for the GWAS data and the reference data
import os
from subprocess import check_output

# set some general defaults
PGC_HITS = "PGC"

POPULATION = "EUR"

# general plotting directory
PLOTS_loc = "PLOTS"

# location to put molQTL results
molQTL_loc = "molQTL_results"

# Check if the directory exists
if not os.path.exists(molQTL_loc):
    # If it doesn't exist, create it
    os.makedirs(molQTL_loc)

# Check if the directory exists
if not os.path.exists(PLOTS_loc):
    # If it doesn't exist, create it
    os.makedirs(PLOTS_loc)

# # regional association plots directory
# REG_PLOTS_loc = PLOTS_loc + "/Regional_Association_Plots"

# # Check if the directory exists
# if not os.path.exists(REG_PLOTS_loc):
#     # If it doesn't exist, create it
#     os.makedirs(REG_PLOTS_loc)

# Reference data directory
REF_loc = "/Users/slaan3/PLINK/references"
print("Checking contents of the reference directory:")
print(check_output(["ls", os.path.join(REF_loc)]).decode("utf8"))

# GWAS data directory
GD_loc = "/Users/slaan3/Library/CloudStorage/GoogleDrive-s.w.vanderlaan@gmail.com/My Drive/Genomics/#Projects/TO_AITION/MR CVD MDD/GWAS"
print("Checking contents of the Google Drive directory:")
print(check_output(["ls", os.path.join(GD_loc)]).decode("utf8"))

# molQTL data directory
NOM_CIS_EQTL_loc = "/Users/slaan3/git/CirculatoryHealth/molqtl/results/version1_aernas1_firstrun/nom_cis_eqtl"
print("Checking contents of the nominal cis-eQTL data directory:")
print(check_output(["ls", os.path.join(NOM_CIS_EQTL_loc)]).decode("utf8"))

PERM_CIS_MQTL_loc = "/Users/slaan3/git/CirculatoryHealth/molqtl/results/perm_cis_mqtl"
print("Checking contents of the permuted cis-mQTL data directory:")
print(check_output(["ls", os.path.join(PERM_CIS_MQTL_loc)]).decode("utf8"))

PERM_TRANS_EQTL_loc = "/Users/slaan3/git/CirculatoryHealth/molqtl/results/version1_aernas1_firstrun/perm_trans_eqtl"
print("Checking contents of the permuted trans-eQTL data directory:")
print(check_output(["ls", os.path.join(PERM_TRANS_EQTL_loc)]).decode("utf8"))

PERM_TRANS_MQTL_loc = (
    "/Users/slaan3/git/CirculatoryHealth/molqtl/results/perm_trans_mqtl"
)
print("Checking contents of the permuted trans-eQTL data directory:")
print(check_output(["ls", os.path.join(PERM_TRANS_MQTL_loc)]).decode("utf8"))

Checking contents of the reference directory:
1000G
GoNL
HRC_r1_1_2016
HRCr11_1000Gp3v5
dbSNP
refgenie_genomes
tcga

Checking contents of the Google Drive directory:
CAC
CIMT
GIGASTROKE
MILLIONHEARTS
PGC

Checking contents of the nominal cis-eQTL data directory:
README.md
tensorqtl_cis_nominal_chr01.cis_qtl_pairs.chr01.parquet
tensorqtl_cis_nominal_chr02.cis_qtl_pairs.chr02.parquet
tensorqtl_cis_nominal_chr03.cis_qtl_pairs.chr03.parquet
tensorqtl_cis_nominal_chr04.cis_qtl_pairs.chr04.parquet
tensorqtl_cis_nominal_chr05.cis_qtl_pairs.chr05.parquet
tensorqtl_cis_nominal_chr06.cis_qtl_pairs.chr06.parquet
tensorqtl_cis_nominal_chr07.cis_qtl_pairs.chr07.parquet
tensorqtl_cis_nominal_chr08.cis_qtl_pairs.chr08.parquet
tensorqtl_cis_nominal_chr09.cis_qtl_pairs.chr09.parquet
tensorqtl_cis_nominal_chr10.cis_qtl_pairs.chr10.parquet
tensorqtl_cis_nominal_chr11.cis_qtl_pairs.chr11.parquet
tensorqtl_cis_nominal_chr12.cis_qtl_pairs.chr12.parquet
tensorqtl_cis_nominal_chr13.cis_qtl_pairs.chr13.parquet

## Load data

We are going to need the list of ROIs and the annotated cis-eQTL data.

### Regions-of-interest

Let's load the 8 regions we are interested in.

In [3]:
# polars.read_excel(
# source: str | BytesIO | Path | BinaryIO | bytes,
# *,
# sheet_id: None = None,
# sheet_name: str,
# engine: Literal['xlsx2csv', 'openpyxl', 'pyxlsb'] | None = None,
# xlsx2csv_options: dict[str, Any] | None = None,
# read_csv_options: dict[str, Any] | None = None,
# schema_overrides: SchemaDict | None = None,
# raise_if_empty: bool = True,
# )

target_regions = pl.read_csv(
    source=os.path.join("targets/regions.csv")
)


In [4]:
target_regions

leadSNP,chromosome,lower_position,upper_position
str,i64,i64,i64
"""7:12263538""",7,11263538,13263538
"""19:4044579""",19,3044579,5044579
"""15:38843887""",15,37843887,39843887
"""7:114059156""",7,113059156,115059156
"""7:2760750""",7,1760750,3760750
"""7:1937261""",7,937261,2937261
"""2:165045101""",2,164045101,166045101
"""2:164915279""",2,163915279,165915279


### cis-eQTL data

We previously parsed the cis-eQTL data into a `.parquet` file which includes annotated results. For details check `` in the `` GitHub repository. 

Here we'll simply load thed annotated data.

In [5]:
# read in data
# https://stackoverflow.com/questions/33813815/how-to-read-a-parquet-file-into-pandas-dataframe

import polars as pl

# annotated cis-eQTLs
sumstats_nom_cis_eqtl = pl.read_parquet(
    source=os.path.join(
        NOM_CIS_EQTL_loc, "tensorqtl_nominal_cis_qtl_pairs.annot.parquet"
    )
)


In [6]:
sumstats_nom_cis_eqtl

EnsemblID,VariantID,tss_distance,CAF_eQTL,ma_samples,ma_count,pval_nominal,Beta,SE,chromosome,position,OtherAlleleA,CodedAlleleB,CAF,AltID,Source,AverageMaximumPosteriorCall,Info,AA_N,AB_N,BB_N,TotalN,MAF,MissingDataProportion,HWE_P
str,str,i32,f32,i32,i32,f64,f32,f32,i64,i64,str,str,f64,str,str,f64,f64,f64,f64,f64,i64,f64,f64,f64
"""ENSG00000187634""","""1:693731""",-240519,0.134185,153,168,0.881492,0.005953,0.039917,1,693731,"""A""","""G""",0.137241,"""rs12238997""","""HRCr11""",0.902046,0.624501,1637.01,451.103,35.569,2124,0.122957,0.000075,0.544841
"""ENSG00000187634""","""1:714596""",-219654,0.038339,48,48,0.834746,-0.015369,0.073635,1,714596,"""T""","""C""",0.0327213,"""rs149887893""","""HRCr11""",0.968278,0.577564,1987.73,135.495,0.677,2124,0.0322165,0.000024,0.272504
"""ENSG00000187634""","""1:715367""",-218883,0.038339,48,48,0.834746,-0.015369,0.073635,1,715367,"""A""","""G""",0.0324859,"""rs12184277""","""HRCr11""",0.975207,0.67196,1976.85,146.512,0.556,2124,0.0347528,0.000019,0.176551
"""ENSG00000187634""","""1:717485""",-216765,0.038339,48,48,0.834746,-0.015369,0.073635,1,717485,"""C""","""A""",0.0324859,"""rs12184279""","""HRCr11""",0.975183,0.670147,1978.35,145.041,0.517,2124,0.0343883,0.000022,0.174817
"""ENSG00000187634""","""1:720381""",-213869,0.038339,48,48,0.834746,-0.015369,0.073635,1,720381,"""G""","""T""",0.0327213,"""rs116801199""","""HRCr11""",0.974476,0.667536,1973.72,149.503,0.691,2124,0.0355205,0.00002,0.110452
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ENSG00000079974""","""22:51220249""",444664,0.035942,43,45,0.863512,-0.004492,0.026119,22,51220249,"""A""","""T""",0.0442561,"""rs9616977""","""1000Gp3v5""",0.913151,0.403798,1832.6,278.562,12.692,2124,0.0715553,0.000034,0.622809
"""ENSG00000079974""","""22:51221190""",445605,0.035144,42,44,0.671267,0.011112,0.026169,22,51221190,"""G""","""A""",0.0400188,"""rs369304721""","""HRCr11""",0.933398,0.443438,1911.83,204.73,7.373,2124,0.0516673,0.000015,0.500544
"""ENSG00000079974""","""22:51221731""",446146,0.035942,43,45,0.863512,-0.004492,0.026119,22,51221731,"""T""","""C""",0.0442561,"""rs115055839""","""HRCr11""",0.911894,0.405944,1826.36,284.126,13.448,2124,0.0732185,0.000016,0.523538


## Extract data

Now we're ready to extract the data for each region in one table.

In [8]:
import polars as pl

# Initialize an empty DataFrame to store the filtered results
filtered_results = pl.DataFrame()

# Loop through each row of df2 and filter df1 based on chromosome and position range
for row in target_regions.iter_rows(named=True):
    chrom = row['chromosome']
    lower_pos = row['lower_position']
    upper_pos = row['upper_position']
    
    # Filter df1 based on the chromosome and position range using polars
    filtered = sumstats_nom_cis_eqtl.filter(
        (sumstats_nom_cis_eqtl['chromosome'] == chrom) & 
        (sumstats_nom_cis_eqtl['position'] >= lower_pos) & 
        (sumstats_nom_cis_eqtl['position'] <= upper_pos)
    )
    
    # Append the filtered rows to the result DataFrame
    filtered_results = filtered_results.vstack(filtered)

# Show the final filtered DataFrame
print(filtered_results)


shape: (621_506, 25)
┌────────────┬────────────┬────────────┬──────────┬───┬────────┬───────────┬────────────┬──────────┐
│ EnsemblID  ┆ VariantID  ┆ tss_distan ┆ CAF_eQTL ┆ … ┆ TotalN ┆ MAF       ┆ MissingDat ┆ HWE_P    │
│ ---        ┆ ---        ┆ ce         ┆ ---      ┆   ┆ ---    ┆ ---       ┆ aProportio ┆ ---      │
│ str        ┆ str        ┆ ---        ┆ f32      ┆   ┆ i64    ┆ f64       ┆ n          ┆ f64      │
│            ┆            ┆ i32        ┆          ┆   ┆        ┆           ┆ ---        ┆          │
│            ┆            ┆            ┆          ┆   ┆        ┆           ┆ f64        ┆          │
╞════════════╪════════════╪════════════╪══════════╪═══╪════════╪═══════════╪════════════╪══════════╡
│ ENSG000001 ┆ 7:11263538 ┆ 327489     ┆ 0.071086 ┆ … ┆ 2124   ┆ 0.0684027 ┆ 0.000001   ┆ 0.610073 │
│ 89043      ┆            ┆            ┆          ┆   ┆        ┆           ┆            ┆          │
│ ENSG000001 ┆ 7:11264536 ┆ 328487     ┆ 0.047923 ┆ … ┆ 2124   ┆ 0.053

In [9]:
# Count the number of rows in the filtered_results DataFrame
num_rows = filtered_results.shape[0]

# Print the number of rows
print(f"Number of rows in filtered_results: {num_rows}")


Number of rows in filtered_results: 621506


In [ ]:
del sumstats_perm_trans_mqtl

## Annotation

Let's also annotate this table with more information.

In [16]:
import mygene
import pandas as pd
import polars as pl

# Initialize mygene client
mg = mygene.MyGeneInfo()

# Convert the polars DataFrame (filtered_results) to a pandas DataFrame
filtered_results_pd = filtered_results.to_pandas()

# Get the unique Ensembl IDs from the polars DataFrame (filtered_results)
ensembl_ids = filtered_results['EnsemblID'].unique().to_list()

# Query mygene to get additional gene information (e.g., symbol, name, etc.)
gene_info = mg.querymany(ensembl_ids, 
                         scopes='ensembl.gene', 
                         fields='symbol,name,alias,type_of_gene,map_location,genomic_pos,entrezgene,HGNC,summary', 
                         species='human')

# Convert the gene info to a pandas DataFrame
gene_info_df = pd.DataFrame(gene_info)

# Select relevant columns
gene_info_df = gene_info_df[['query', 'symbol', 'name', 'alias', 'type_of_gene', 'map_location', 'genomic_pos', 'entrezgene', 'HGNC', 'summary']]

# Convert the pandas DataFrame back to polars DataFrame
gene_info_df

INFO:biothings.client:querying 1-176...
INFO:biothings.client:done.
INFO:biothings.client:Finished.


,query,symbol,name,alias,type_of_gene,map_location,genomic_pos,entrezgene,HGNC,summary
0,ENSG00000140320,BAHD1,bromo adjacent homology domain containing 1,NaN,protein-coding,15q15.1,"{'chr': '15', 'end': 40468236, 'ensemblgene': ...",22893,29153,Enables chromatin binding activity. Involved i...
1,ENSG00000064961,HMG20B,high mobility group 20B,"[BRAF25, BRAF35, HMGX2, HMGXB2, PP7706, SMARCE...",protein-coding,19p13.3,"{'chr': '19', 'end': 3579088, 'ensemblgene': '...",10362,5002,Predicted to enable DNA binding activity. Pred...
2,ENSG00000065717,TLE2,"TLE family member 2, transcriptional corepressor","[ESG, ESG2, GRG2]",protein-coding,19p13.3,"{'chr': '19', 'end': 3047635, 'ensemblgene': '...",7089,11838,Enables transcription corepressor activity. In...
3,ENSG00000146802,TMEM168,transmembrane protein 168,NaN,protein-coding,7q31.1,"{'chr': '7', 'end': 112790423, 'ensemblgene': ...",64418,25826,Located in transport vesicle. [provided by All...
4,ENSG00000073067,CYP2W1,cytochrome P450 family 2 subfamily W member 1,NaN,protein-coding,7p22.3,"{'chr': '7', 'end': 989640, 'ensemblgene': 'EN...",54905,20243,This gene encodes a member of the cytochrome P...
...,...,...,...,...,...,...,...,...,...,...
171,ENSG00000166068,SPRED1,sprouty related EVH1 domain containing 1,"[LGSS, NFLS, PPP1R147, hSpred1, spred-1]",protein-coding,15q14,"{'chr': '15', 'end': 38357249, 'ensemblgene': ...",161742,20249,The protein encoded by this gene is a member o...
172,ENSG00000130255,RPL36,ribosomal protein L36,"[L36, eL36]",protein-coding,19p13.3,"{'chr': '19', 'end': 5691875, 'ensemblgene': '...",25873,13631,"Ribosomes, the organelles that catalyze protei..."
173,ENSG00000105229,PIAS4,protein inhibitor of activated STAT 4,"[PIAS-gamma, PIASY, Piasg, ZMIZ6]",protein-coding,19p13.3,"{'chr': '19', 'end': 4039386, 'ensemblgene': '...",51588,17002,Enables SUMO ligase activity and ubiquitin pro...
174,ENSG00000128891,CCDC32,coiled-coil domain containing 32,"[C15orf57, CFNDS]",protein-coding,15q15.1,"{'chr': '15', 'end': 40565054, 'ensemblgene': ...",90416,28295,Involved in head development. [provided by All...


In [20]:
# Merge the original pandas DataFrame (filtered_results_pd) with the gene information DataFrame on 'EnsemblID'
filtered_results_annotated = filtered_results_pd.merge(gene_info_df, left_on='EnsemblID', right_on='query', how='left')

# Drop the 'query' column (since it's the same as 'EnsemblID')
filtered_results_annotated.drop(columns=['query'], inplace=True)

# Display the annotated DataFrame
print(filtered_results_annotated)

              EnsemblID    VariantID  tss_distance  CAF_eQTL  ma_samples  \
0       ENSG00000189043   7:11263538        327489  0.071086          84   
1       ENSG00000189043   7:11264536        328487  0.047923          56   
2       ENSG00000189043   7:11264670        328621  0.075879          88   
3       ENSG00000189043   7:11264692        328643  0.521565         455   
4       ENSG00000189043   7:11265421        329372  0.598243         404   
...                 ...          ...           ...       ...         ...   
621501  ENSG00000136546  2:165913564       -593964  0.646965         362   
621502  ENSG00000136546  2:165914241       -593287  0.162939         186   
621503  ENSG00000136546  2:165914353       -593175  0.227636         251   
621504  ENSG00000136546  2:165914416       -593112  0.234026         259   
621505  ENSG00000136546  2:165914802       -592726  0.137380         155   

        ma_count  pval_nominal      Beta        SE  chromosome  ...     HWE_P  \
0     

In [21]:
# Count the number of rows in filtered_results_annotated_pd
num_rows = filtered_results_annotated_pd.shape[0]

# Print the number of rows
print(f"Number of rows in filtered_results_annotated_pd: {num_rows}")


Number of rows in filtered_results_annotated_pd: 621506


In [22]:
# Display the headers of the DataFrame
print(filtered_results_annotated_pd.columns)


Index(['EnsemblID', 'VariantID', 'tss_distance', 'CAF_eQTL', 'ma_samples',
       'ma_count', 'pval_nominal', 'Beta', 'SE', 'chromosome', 'position',
       'OtherAlleleA', 'CodedAlleleB', 'CAF', 'AltID', 'Source',
       'AverageMaximumPosteriorCall', 'Info', 'AA_N', 'AB_N', 'BB_N', 'TotalN',
       'MAF', 'MissingDataProportion', 'HWE_P', 'symbol', 'name', 'alias',
       'type_of_gene', 'map_location', 'genomic_pos', 'entrezgene', 'HGNC',
       'summary'],
      dtype='object')


## Saving
 Let's save the annotated table.

In [24]:
# Specify the order of columns if needed
columns_order = ['EnsemblID', 'symbol', 
'VariantID', 'tss_distance', 'CAF_eQTL', 'ma_samples',
'ma_count', 'pval_nominal', 'Beta', 'SE', 'chromosome', 'position',
'OtherAlleleA', 'CodedAlleleB', 'CAF', 'AltID', 'Source',
'AverageMaximumPosteriorCall', 'Info', 'AA_N', 'AB_N', 'BB_N', 'TotalN',
'MAF', 'MissingDataProportion', 'HWE_P', 
'name', 'alias', 'type_of_gene', 'map_location', 'genomic_pos', 'entrezgene', 'HGNC', 'summary']
# Ensure these columns exist in your DataFrame

# Save the DataFrame with specified columns
filtered_results_annotated_pd.to_csv(
    os.path.join(
        molQTL_loc, 'target_regions.nom_cis_eqtl_annotated.txt.gz'),
    sep='\t',
    index=False,
    compression='gzip',
    columns=columns_order
)
